In [6]:
from sklearn.linear_model import LinearRegression,RidgeCV,LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
data=pd.read_csv('cardekho_dataset.csv')
data

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,19537,Hyundai i10,Hyundai,i10,9,10723,Dealer,Petrol,Manual,19.81,1086,68.05,5,250000
15407,19540,Maruti Ertiga,Maruti,Ertiga,2,18000,Dealer,Petrol,Manual,17.50,1373,91.10,7,925000
15408,19541,Skoda Rapid,Skoda,Rapid,6,67000,Dealer,Diesel,Manual,21.14,1498,103.52,5,425000
15409,19542,Mahindra XUV500,Mahindra,XUV500,5,3800000,Dealer,Diesel,Manual,16.00,2179,140.00,7,1225000


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         15411 non-null  int64  
 1   car_name           15411 non-null  object 
 2   brand              15411 non-null  object 
 3   model              15411 non-null  object 
 4   vehicle_age        15411 non-null  int64  
 5   km_driven          15411 non-null  int64  
 6   seller_type        15411 non-null  object 
 7   fuel_type          15411 non-null  object 
 8   transmission_type  15411 non-null  object 
 9   mileage            15411 non-null  float64
 10  engine             15411 non-null  int64  
 11  max_power          15411 non-null  float64
 12  seats              15411 non-null  int64  
 13  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 1.6+ MB


In [9]:
data.isna().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [10]:
categorical_columns=[]
for column in data.columns:
    if data[column].dtype=='object':
        categorical_columns.append(column)
categorical_columns

['car_name', 'brand', 'model', 'seller_type', 'fuel_type', 'transmission_type']

### As there are large number of unique values in the columns we have to use categpry encoders to convert them into numerical data

In [11]:
X=data.drop(['selling_price'],axis=1)
y=data[['selling_price']]

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
11210,14238,Volvo XC60,Volvo,XC60,7,70252,Dealer,Diesel,Automatic,11.20,2400,215.00,5
1347,1731,Tata Tiago,Tata,Tiago,2,10000,Individual,Petrol,Manual,23.84,1199,84.00,5
10363,13218,Mercedes-Benz CLS,Mercedes-Benz,CLS,2,6000,Dealer,Diesel,Automatic,19.00,1950,241.30,5
316,403,Honda City,Honda,City,7,63000,Dealer,Petrol,Manual,17.80,1497,117.30,5
10638,13550,Hyundai i10,Hyundai,i10,10,80292,Dealer,Petrol,Manual,20.36,1197,78.90,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,6581,Maruti Ertiga,Maruti,Ertiga,7,127731,Dealer,Diesel,Manual,20.77,1248,88.80,7
13418,17029,Volkswagen Vento,Volkswagen,Vento,11,59000,Dealer,Petrol,Manual,16.09,1598,103.20,5
5390,6839,Maruti Wagon R,Maruti,Wagon R,7,20000,Individual,Petrol,Manual,20.51,998,67.04,5
860,1104,Hyundai i20,Hyundai,i20,2,15000,Dealer,Petrol,Manual,18.60,1197,81.86,5


In [13]:
scaler=StandardScaler()
import category_encoders as ce 

In [14]:
encoder=ce.TargetEncoder(cols=categorical_columns)
encoder

TargetEncoder(cols=['car_name', 'brand', 'model', 'seller_type', 'fuel_type',
                    'transmission_type'])